# Q1: Whisper-Small Fine-Tuning on JoshTalks Hindi Data
**JoshTalks ASR Research - Ready-to-Run Colab Notebook**

## What This Does
1. Upload your `FT Data.xlsx` (104 recordings, ~22 hours of Hindi)
2. Fixes GCP URLs (old base path -> new `upload_goai` path)
3. Downloads audio + transcription from corrected URLs
4. Fine-tunes `openai/whisper-small` on this data
5. Evaluates on FLEURS Hindi test set
6. **Outputs the WER value to fill into FT Result.xlsx**

### Prerequisites
- Google Colab with **GPU runtime** (T4 or better)
- `FT Data.xlsx` file from the assignment

### Expected Time: ~40-50 minutes on T4 GPU

## Step 1: Install Dependencies and Check GPU

In [ ]:
!pip install -q --upgrade transformers torch datasets==2.14.6 evaluate jiwer librosa soundfile tqdm pandas numpy matplotlib openpyxl accelerate

In [ ]:
import torch
import json
import os
import re
import unicodedata
import numpy as np
import pandas as pd
import requests
import librosa
import soundfile as sf
from datetime import datetime
from tqdm import tqdm
from pathlib import Path

print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f'GPU Memory: {mem_gb:.1f} GB')
else:
    print('WARNING: No GPU! Go to Runtime > Change runtime type > GPU')

## Step 2: Upload FT Data.xlsx
Run the cell below - it will open a file picker. Select your `FT Data.xlsx` file.

In [ ]:
from google.colab import files

print('Please upload your FT Data.xlsx file:')
uploaded = files.upload()

xlsx_file = None
for fname in uploaded:
    if fname.endswith('.xlsx'):
        xlsx_file = fname
        break

if xlsx_file:
    print(f'\nLoaded: {xlsx_file}')
    df = pd.read_excel(xlsx_file, engine='openpyxl')
    print(f'Recordings: {len(df)}')
    print(f'Columns: {list(df.columns)}')
    total_hrs = df['duration'].sum() / 3600
    print(f'Total duration: {total_hrs:.1f} hours')
    display(df.head(3))
else:
    raise FileNotFoundError('No .xlsx file found in upload!')

## Step 3: Fix GCP URLs and Download Data

The original URLs use `joshtalks-data-collection/hq_data/hi/` which no longer works.
Per the task instructions, we replace the base path with `upload_goai/`.

**Example:**
- Old: `https://storage.googleapis.com/joshtalks-data-collection/hq_data/hi/967179/825780_audio.wav`
- New: `https://storage.googleapis.com/upload_goai/967179/825780_audio.wav`

In [ ]:
OLD_BASE = 'storage.googleapis.com/joshtalks-data-collection/hq_data/hi/'
NEW_BASE = 'storage.googleapis.com/upload_goai/'

def fix_url(url):
    if pd.isna(url) or not isinstance(url, str):
        return None
    url = str(url).strip()
    if OLD_BASE in url:
        url = url.replace(OLD_BASE, NEW_BASE)
    return url

# Fix all URL columns
url_cols = ['rec_url_gcp', 'transcription_url_gcp', 'metadata_url_gcp']
for col in url_cols:
    if col in df.columns:
        df[col + '_fixed'] = df[col].apply(fix_url)

# Show example
print('URL Fix Examples:')
if 'rec_url_gcp' in df.columns:
    print(f'  Old: {df["rec_url_gcp"].iloc[0]}')
    print(f'  New: {df["rec_url_gcp_fixed"].iloc[0]}')
print()
if 'transcription_url_gcp' in df.columns:
    print(f'  Old: {df["transcription_url_gcp"].iloc[0]}')
    print(f'  New: {df["transcription_url_gcp_fixed"].iloc[0]}')

In [ ]:
DATA_DIR = '/content/ft_data'
AUDIO_DIR = os.path.join(DATA_DIR, 'audio')
TRANS_DIR = os.path.join(DATA_DIR, 'transcriptions')
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANS_DIR, exist_ok=True)

def download_file(url, save_path, timeout=120):
    try:
        if not url or not str(url).startswith('http'):
            return False
        resp = requests.get(str(url), timeout=timeout, stream=True)
        if resp.status_code == 200:
            with open(save_path, 'wb') as f:
                for chunk in resp.iter_content(8192):
                    f.write(chunk)
            return os.path.getsize(save_path) > 100
        else:
            return False
    except Exception as e:
        return False

results = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc='Downloading'):
    rec_id = str(int(row['recording_id']))
    user_id = str(int(row['user_id']))

    audio_path = os.path.join(AUDIO_DIR, f'{rec_id}.wav')
    trans_path = os.path.join(TRANS_DIR, f'{rec_id}.json')

    # Use fixed URLs
    audio_url = row.get('rec_url_gcp_fixed', None)
    trans_url = row.get('transcription_url_gcp_fixed', None)

    audio_ok = os.path.exists(audio_path) and os.path.getsize(audio_path) > 100
    if not audio_ok:
        audio_ok = download_file(audio_url, audio_path)

    trans_ok = os.path.exists(trans_path) and os.path.getsize(trans_path) > 10
    if not trans_ok:
        trans_ok = download_file(trans_url, trans_path)

    results.append({
        'recording_id': rec_id,
        'user_id': user_id,
        'duration': row.get('duration', 0),
        'audio_path': audio_path if audio_ok else None,
        'trans_path': trans_path if trans_ok else None,
        'audio_ok': audio_ok,
        'trans_ok': trans_ok,
    })

dl_df = pd.DataFrame(results)
both_ok = (dl_df['audio_ok'] & dl_df['trans_ok']).sum()
audio_only = dl_df['audio_ok'].sum()
trans_only = dl_df['trans_ok'].sum()
print(f'\nAudio downloaded: {audio_only}/{len(df)}')
print(f'Transcriptions downloaded: {trans_only}/{len(df)}')
print(f'Both available: {both_ok}/{len(df)}')

dl_df = dl_df[dl_df['audio_ok'] & dl_df['trans_ok']].reset_index(drop=True)
print(f'\nUsing {len(dl_df)} recordings for training')

if len(dl_df) == 0:
    raise RuntimeError('No recordings downloaded! Check if URLs are accessible.')

## Step 4: Extract Transcription Text and Build Training Data

In [ ]:
def normalize_text(text):
    text = unicodedata.normalize('NFKC', text)
    text = text.replace('\u200b', '').replace('\u200c', '').replace('\u200d', '')
    text = re.sub(r'[^\w\s\u0900-\u097F]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def load_transcription(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, str):
            return normalize_text(data)
        elif isinstance(data, dict):
            for key in ['text', 'transcription', 'transcript', 'content']:
                if key in data:
                    return normalize_text(str(data[key]))
            if 'segments' in data:
                texts = [s.get('text', '') for s in data['segments']]
                return normalize_text(' '.join(texts))
            for v in data.values():
                if isinstance(v, str) and len(v) > 5:
                    return normalize_text(v)
        elif isinstance(data, list):
            if data and isinstance(data[0], dict):
                texts = [item.get('text', '') for item in data]
                return normalize_text(' '.join(texts))
            return normalize_text(' '.join(str(x) for x in data))
    except Exception as e:
        print(f'  Error reading {json_path}: {e}')
    return None

samples = []
for idx, row in tqdm(dl_df.iterrows(), total=len(dl_df), desc='Loading transcriptions'):
    text = load_transcription(row['trans_path'])
    if text and len(text) > 5:
        samples.append({
            'recording_id': row['recording_id'],
            'audio_path': row['audio_path'],
            'text': text,
            'duration': row['duration'],
        })

train_df = pd.DataFrame(samples)
print(f'\nTraining samples with valid text: {len(train_df)}')
total_h = train_df['duration'].sum() / 3600
avg_len = train_df['text'].str.len().mean()
print(f'Total hours: {total_h:.1f}h')
print(f'Avg text length: {avg_len:.0f} chars')
print(f'\nSample text:')
print(f'  {train_df["text"].iloc[0][:200]}...')

## Step 5: Setup Whisper Model and Dataset

In [ ]:
from torch.utils.data import Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from dataclasses import dataclass
from typing import Any, Dict, List, Union

MODEL_NAME = 'openai/whisper-small'
LANGUAGE = 'hi'
TASK = 'transcribe'

EPOCHS = 2
BATCH_SIZE = 2
GRADIENT_ACCUMULATION = 8
LEARNING_RATE = 1e-5
WARMUP_STEPS = 50
FP16 = True
MAX_AUDIO_SECONDS = 30
SEED = 42

OUTPUT_DIR = '/content/outputs/q1'
MODEL_DIR = '/content/outputs/q1/finetuned_model'
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
torch.manual_seed(SEED)
np.random.seed(SEED)

processor = WhisperProcessor.from_pretrained(MODEL_NAME)
print(f'Processor loaded: {MODEL_NAME}')

In [ ]:
class JoshTalksASRDataset(Dataset):

    def __init__(self, metadata_df, processor, max_seconds=30):
        self.processor = processor
        self.max_seconds = max_seconds
        self.sr = 16000

        self.samples = []
        for _, row in tqdm(metadata_df.iterrows(), total=len(metadata_df), desc='Segmenting audio'):
            try:
                audio, sr = librosa.load(row['audio_path'], sr=self.sr, mono=True)
                text = row['text']
                dur = len(audio) / self.sr

                if dur <= max_seconds:
                    self.samples.append({'audio': audio, 'text': text})
                else:
                    words = text.split()
                    chunk_size = int(max_seconds * self.sr)
                    n_chunks = int(np.ceil(len(audio) / chunk_size))
                    words_per_chunk = max(1, len(words) // n_chunks)

                    for i in range(n_chunks):
                        start = i * chunk_size
                        end = min((i + 1) * chunk_size, len(audio))
                        chunk_audio = audio[start:end]

                        w_start = i * words_per_chunk
                        w_end = min((i + 1) * words_per_chunk, len(words))
                        if i == n_chunks - 1:
                            w_end = len(words)
                        chunk_text = ' '.join(words[w_start:w_end])

                        if chunk_text.strip() and len(chunk_audio) > self.sr:
                            self.samples.append({'audio': chunk_audio, 'text': chunk_text})
            except Exception as e:
                print(f'  Skipping {row["recording_id"]}: {e}')

        print(f'  Created {len(self.samples)} training segments')

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        input_features = self.processor(
            sample['audio'], sampling_rate=16000, return_tensors='pt'
        ).input_features[0]
        labels = self.processor.tokenizer(sample['text']).input_ids
        return {'input_features': input_features, 'labels': labels}


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features):
        input_features = [{'input_features': f['input_features']} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors='pt')

        label_features = [{'input_ids': f['labels']} for f in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors='pt')

        labels = labels_batch['input_ids'].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        bos_id = self.processor.tokenizer.bos_token_id
        if (labels[:, 0] == bos_id).all().cpu().item():
            labels = labels[:, 1:]

        batch['labels'] = labels
        return batch

print('Dataset class ready.')

In [ ]:
from sklearn.model_selection import train_test_split

train_split, val_split = train_test_split(train_df, test_size=0.1, random_state=SEED)
print(f'Train: {len(train_split)} recordings | Val: {len(val_split)} recordings')

print('\nBuilding training dataset (loading + segmenting audio)...')
train_dataset = JoshTalksASRDataset(train_split, processor, max_seconds=MAX_AUDIO_SECONDS)

print('\nBuilding validation dataset...')
val_dataset = JoshTalksASRDataset(val_split, processor, max_seconds=MAX_AUDIO_SECONDS)

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

print(f'\nTrain segments: {len(train_dataset)}')
print(f'Val segments: {len(val_dataset)}')

## Step 6: Fine-Tune Whisper
Main training step. Takes ~30-40 minutes on T4 GPU.

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
model.generation_config.language = LANGUAGE
model.generation_config.task = TASK
model.generation_config.forced_decoder_ids = None

wer_metric = evaluate.load('wer')

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)
    return {'wer': wer}

param_count = sum(p.numel() for p in model.parameters()) / 1e6
print(f'Model loaded: {param_count:.1f}M params')

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=EPOCHS,
    fp16=FP16,
    eval_strategy='steps',
    eval_steps=100,
    save_steps=100,
    logging_steps=25,
    predict_with_generate=True,
    generation_max_length=225,
    report_to=['none'],
    load_best_model_at_end=True,
    metric_for_best_model='wer',
    greater_is_better=False,
    save_total_limit=2,
    seed=SEED,
    dataloader_num_workers=2,
    remove_unused_columns=False,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # tokenizer=processor.tokenizer,  # This argument seems to be unexpected in this version
)

eff_batch = BATCH_SIZE * GRADIENT_ACCUMULATION
print(f'Trainer ready.')
print(f'  Epochs: {EPOCHS}')
print(f'  Effective batch size: {eff_batch}')
print(f'  Learning rate: {LEARNING_RATE}')

In [ ]:
import time

print('Starting fine-tuning...')
start_time = time.time()

train_result = trainer.train()

train_time = time.time() - start_time
print(f'\nTraining completed in {train_time / 60:.1f} minutes')
print(f'Final train loss: {train_result.training_loss:.4f}')

trainer.save_model(MODEL_DIR)
processor.save_pretrained(MODEL_DIR)
print(f'Model saved to {MODEL_DIR}')

## Step 7: Training Loss Curves

In [ ]:
import matplotlib.pyplot as plt

log_history = trainer.state.log_history

train_losses = [(l['step'], l['loss']) for l in log_history if 'loss' in l]
eval_losses = [(l['step'], l['eval_loss']) for l in log_history if 'eval_loss' in l]
eval_wers = [(l['step'], l['eval_wer']) for l in log_history if 'eval_wer' in l]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

if train_losses:
    steps, losses = zip(*train_losses)
    axes[0].plot(steps, losses, 'b-', alpha=0.7, linewidth=2)
    axes[0].set_xlabel('Step')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training Loss', fontweight='bold')
    axes[0].grid(alpha=0.3)

if eval_losses:
    steps, losses = zip(*eval_losses)
    axes[1].plot(steps, losses, 'r-o', alpha=0.7, linewidth=2)
    axes[1].set_xlabel('Step')
    axes[1].set_ylabel('Loss')
    axes[1].set_title('Validation Loss', fontweight='bold')
    axes[1].grid(alpha=0.3)

if eval_wers:
    steps, wers = zip(*eval_wers)
    axes[2].plot(steps, wers, 'g-o', alpha=0.7, linewidth=2)
    axes[2].set_xlabel('Step')
    axes[2].set_ylabel('WER (%)')
    axes[2].set_title('Validation WER', fontweight='bold')
    axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_curves.png'), dpi=150, bbox_inches='tight')
plt.show()
print('Training curves saved.')

## Step 8: Evaluate on FLEURS Hindi Test Set
Computes the final WER - the value you need for FT Result.xlsx.

In [ ]:
from datasets import load_dataset
import jiwer

NUM_EVAL = 200

def evaluate_model(model_to_eval, processor, num_samples=200, label='model'):
    device = next(model_to_eval.parameters()).device
    model_to_eval.eval()

    fleurs_test = load_dataset('google/fleurs', 'hi_in', split='test')
    n = min(num_samples, len(fleurs_test))

    forced_ids = processor.get_decoder_prompt_ids(language='hi', task='transcribe')

    references = []
    predictions = []

    for i in tqdm(range(n), desc=f'Evaluating {label}'):
        sample = fleurs_test[i]
        audio = np.array(sample['audio']['array'], dtype=np.float32)
        sr = sample['audio']['sampling_rate']
        if sr != 16000:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=16000)

        input_features = processor(
            audio, sampling_rate=16000, return_tensors='pt'
        ).input_features.to(device)

        with torch.no_grad():
            predicted_ids = model_to_eval.generate(
                input_features,
                forced_decoder_ids=forced_ids,
                max_new_tokens=225,
            )

        pred = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        references.append(sample['transcription'])
        predictions.append(pred)

    wer_val = jiwer.wer(references, predictions)
    cer_val = jiwer.cer(references, predictions)

    return {
        'label': label,
        'corpus_wer': round(wer_val, 4),
        'corpus_cer': round(cer_val, 4),
        'num_samples': n,
        'timestamp': datetime.now().isoformat(),
    }

print('Evaluation function ready.')

In [ ]:
print('Evaluating fine-tuned Whisper on FLEURS Hindi test...')
ft_metrics = evaluate_model(
    model, processor, num_samples=NUM_EVAL, label='FT Whisper Small'
)

print('\nEvaluating pretrained Whisper baseline...')
baseline_model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
baseline_model.generation_config.language = LANGUAGE
baseline_model.generation_config.task = TASK
device = next(model.parameters()).device
baseline_model = baseline_model.to(device)

baseline_metrics = evaluate_model(
    baseline_model, processor, num_samples=NUM_EVAL,
    label='Whisper Small Pretrained'
)

del baseline_model
torch.cuda.empty_cache()
print('\nEvaluation complete!')

## Step 9: YOUR RESULTS - Fill Into FT Result.xlsx

In [ ]:
bl_wer = baseline_metrics['corpus_wer']
ft_wer = ft_metrics['corpus_wer']
bl_cer = baseline_metrics['corpus_cer']
ft_cer = ft_metrics['corpus_cer']
improvement = bl_wer - ft_wer

print()
print('=' * 60)
print('  RESULTS FOR FT Result.xlsx')
print('=' * 60)
print()
print(f'  Whisper Small (Pretrained)  WER = {bl_wer:.4f}')
print(f'  FT Whisper Small (yours)    WER = {ft_wer:.4f}  <-- FILL THIS')
print()
print(f'  WER Improvement: {improvement:.4f}')
print(f'  CER: Baseline={bl_cer:.4f}  Fine-tuned={ft_cer:.4f}')
print()
print('=' * 60)
print()
print('  Open FT Result.xlsx and fill in:')
print(f'  Row: FT Whisper Small (yours)')
print(f'  Hindi column: {ft_wer:.4f}')
print()
print('=' * 60)

# Save results
with open(os.path.join(OUTPUT_DIR, 'baseline_metrics.json'), 'w') as f:
    json.dump(baseline_metrics, f, indent=2)
with open(os.path.join(OUTPUT_DIR, 'finetuned_metrics.json'), 'w') as f:
    json.dump(ft_metrics, f, indent=2)

print(f'\nMetrics saved to {OUTPUT_DIR}/')

## Step 10: Download Results

In [ ]:
from google.colab import files

for fname in ['finetuned_metrics.json', 'baseline_metrics.json', 'training_curves.png']:
    fpath = os.path.join(OUTPUT_DIR, fname)
    if os.path.exists(fpath):
        files.download(fpath)
        print(f'Downloaded {fname}')

print('\nAll done! Fill the WER value shown above into FT Result.xlsx.')

In [ ]:
!zip -r outputs.zip outputs/

In [ ]:
!zip -r ft_data.zip ft_data